In [1]:
import pandas as pd
import numpy as np

from utils.data_utils import DataUtils

In [2]:
pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_colwidth", 100)

----

#### Download data

In [3]:
DataUtils.download_data("sql/select_comments.sql", "comments")

In [5]:
DataUtils.download_data("sql/select_tags.sql", "tags")

In [6]:
comments = pd.read_parquet("data/initial_data/comments.parquet")
comments.query("not comment.isna() & not place_id.isna()", inplace=True)

In [7]:
tags = pd.read_parquet("data/initial_data/tags.parquet")

In [8]:
df = pd.DataFrame().from_records(comments.comment.values)

In [11]:
df.insert(0, "place_id", comments.place_id.values)

In [12]:
df = df.merge(tags, left_on="place_id", right_on="place_id", how="left")

In [13]:
df.drop("id", axis=1, inplace=True)

In [14]:
df.to_parquet("data/intermediate_data/comments.parquet")

-----

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("data/intermediate_data/comments.parquet")

In [3]:
df.sample(5)

,place_id,review_body,review_date,review_likes,review_rating,review_tag,review_title,tag
3594,100,Замечательное и очень уютное место! Хорошая ку...,23 февраля 2018 г.,0.0,5.0,"[Бизнес, RU]",Турнир по боулингу,Бары и клубы
1041,2,Очень красивый и многолюдный мост. Гуляя по ис...,21 июня 2014 г.,2.0,5.0,"[Бизнес, RU]",Визитная карточка города,Мосты
599,1,"Я живу в Тюмени, и можно много рассказывать о ...",9 июня 2014 г.,4.0,5.0,"[Бизнес, RU]",Чудесное место,Пирсы и пляжные променады
2066,11,Я считаю музеи обязательными для посещения: го...,9 октября 2017 г.,1.0,3.0,"[С друзьями, RU]","Не Лувр, конечно, но сходить надо обязательно",Специализированные музеи
1696,5,"Думаю, таким образом много денег не насобираеш...",27 января 2015 г.,1.0,3.0,"[Семейный отдых, RU]",Ни к чему это,Культурные объекты и достопримечательности


In [4]:
test = df.review_body[0]
test1 = df.review_body[2]
test2 = df.review_body[3]
test3 = df.review_body[124]
test4 = df.review_body[412]

print(test, "\n")
print(test1, "\n")
print(test2, "\n")
print(test3, "\n")

Красивая набережная с барельефами отражающими историю города Тюмень Красивые виды Зимой строят специальную горку для фристайла и сноуборда Можно посмотреть соревнования а можно просто погулять 

Отличное место для отдыха, прогулок и даже знакомства с историей города и Сибири. Очень круто, что скульптурные композиции, расположенные вдоль прогулочного маршрута, знакомят вас с историей региона, города, выдающихся личностей, а не как это сейчас популярно делать во многих городах - ни к чему не относящиеся, и ни о чем не говорящие, скульптуры а-ля городовой, кот/пёс/лис/конь на удачу и т.п.
Обязательно надо прогуляться по набережной днем, а затем в вечернее время, когда включается архитектурно-художественная подсветка. Идея сделать мост не просто красиво, а в форме струги, придаёт масштаб всей концепции историзма набережной Туры. 

Красивая набережная, можно покататься на самокате, гуляет много народу, мост влюбленных вечером красиво светится 

Отличное место во всех смыслах! Погулять пароч

In [5]:
from models import mt5_ml_xls
from models import roberta

/home/dmitry/.cache/pypoetry/virtualenvs/lewart-tags-model-dMjvlOT--py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model = roberta()

In [7]:
# TODO() кластеризация по эмбендингам

In [25]:
review = test3
proba = 0.3
questions = ["Чем заняться?", "Какой?", "Почему?", "В какое время?", "Зачем?"]

In [26]:
from pprint import pprint
pprint(review)
print("\n")
for question in questions:
    print(question, ": ", model.get_tags(review, question, proba=proba))

('Отличное место во всех смыслах! Погулять парочкам, покататься на '
 'велосипедах, заняться пробежкой, прогуляться с родителями, пообщаться с '
 'друзьями... И так далее... И это всё она, набережная Тюмени, прекрасна в '
 'любое время суток, приезжайте и убедитесь сами 😊')


Чем заняться? :  ['пробежка']
Какой? :  []
Почему? :  []
В какое время? :  ['любое время суток']
Зачем? :  []


In [87]:
answer = model.answer(review, "Чем заняться?", proba=proba)
print(answer)

для отдыха, прогулок


In [88]:
answer_cleared = model._clear_stopwords(answer)
print(answer_cleared)

['отдыха', 'прогулок']


In [89]:
parts = []
for token in answer_cleared:
    parts.append(model._get_part_of_words(token))

In [90]:
parts

[{'отдыха': {'lemma': 'отдых', 'pos': 'NOUN', 'dep': 'ROOT'}},
 {'прогулок': {'lemma': 'прогулка', 'pos': 'NOUN', 'dep': 'ROOT'}}]